## Analiza koszyka rynkowego

<img src="https://thumbor.forbes.com/thumbor/960x0/https%3A%2F%2Fblogs-images.forbes.com%2Fmarciaturner%2Ffiles%2F2018%2F01%2FWegmans-Produce-1.jpg" width="800px">

Częste zestawy przedmiotów za pośrednictwem algorytmu Apriori
Funkcja Apriori do wyodrębniania częstych zestawów elementów do eksploracji reguł skojarzeń
Mamy zbiór danych centrum handlowego z 7500 transakcjami różnych klientów kupujących różne produkty w sklepie.
Musimy znaleźć korelacje między różnymi przedmiotami w sklepie. abyśmy wiedzieli, czy klient kupuje jabłko,
banan i mango. jaka jest następna pozycja, Klient byłby zainteresowany zakupem w sklepie.

## Przegląd
Apriori to popularny algorytm do wyodrębniania częstych zestawów elementów z aplikacjami do uczenia się reguł asocjacyjnych. Algorytm apriori został zaprojektowany do działania na bazach danych zawierających transakcje, np. Zakupy klientów sklepu. Zestaw elementów jest uważany za „częsty”, jeśli spełnia określony przez użytkownika próg wsparcia. Na przykład,
jeżeli próg wsparcia jest ustawiony na 0,5 (50%), częsty zbiór pozycji definiuje się jako zbiór pozycji, które występują razem w co najmniej 50% wszystkich transakcji w bazie danych.

**Importowanie bibliotek**

In [ ]:
# do podstawowych operacji
import numpy as np
import pandas as pd

# do wizualizacji
import matplotlib.pyplot as plt
import squarify
import seaborn as sns
plt.style.use('fivethirtyeight')

# do analizy koszyka rynkowego
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


**Importowanie zbioru danych**

In [ ]:
# czytanie zbioru danych

data1 = pd.read_csv('../input/groceries/groceries - groceries.csv', header = None)

# sprawdźmy kształt zbioru danych
data1.shape

In [ ]:
# sprawdzenie nagłówek danych

data1.head()

In [ ]:
# sprawdzanie ogona danych

data1.tail()

In [ ]:
data = data1.fillna('')
data

In [ ]:
# sprawdzanie losowych wpisów w danych

data.sample(10)

In [ ]:
# opiszmy zbiór danych

data.describe()

## Wizualizacje danych

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud

plt.rcParams['figure.figsize'] = (15, 15)
wordcloud = WordCloud(background_color = 'white', width = 900,  height = 900, max_words = 10).generate(str(data[1]))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Najpopularniejsze przedmioty',fontsize = 20)
plt.show()

In [ ]:
# patrząc na częstotliwość najpopularniejszych pozycji

plt.rcParams['figure.figsize'] = (18, 7)
color = plt.cm.copper(np.linspace(0, 1, 40))
data[1].value_counts().head(40).plot.bar(color = color)
plt.title('przegląd na częstotliwość najpopularniejszych pozycji', fontsize = 20)
plt.xticks(rotation = 90 )
plt.grid()
plt.show()

In [ ]:

y = data[1].value_counts().head(50).to_frame()
y.index

In [ ]:
# rysowanie mapy drzew

plt.rcParams['figure.figsize'] = (20, 20)
color = plt.cm.cool(np.linspace(0, 1, 50))
squarify.plot(sizes = y.values, label = y.index, alpha=.8, color = color)
plt.title('Mapa drzewa dla popularnych pozycji')
plt.axis('off')
plt.show()

In [ ]:
data['Food'] = 'Food'
food = data.truncate(before = 1, after = 16)


import networkx as nx

food = nx.from_pandas_edgelist(food, source = 'Food', target = 1, edge_attr = True)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (15, 15)
pos = nx.spring_layout(food)
color = plt.cm.Wistia(np.linspace(0, 15, 1))
nx.draw_networkx_nodes(food, pos, node_size = 15000, node_color = color)
nx.draw_networkx_edges(food, pos, width = 3, alpha = 0.6, edge_color = 'black')
nx.draw_networkx_labels(food, pos, font_size = 15, font_family = 'sans-serif')
plt.axis('off')
plt.grid()
plt.title('10 pierwszych wyborów', fontsize = 40)
plt.show()

In [ ]:
data['secondchoice'] = 'Second Choice'
secondchoice = data.truncate(before = -1, after = 15)
secondchoice = nx.from_pandas_edgelist(secondchoice, source = 'Food', target = 2, edge_attr = True)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.rcParams['figure.figsize'] = (20, 20)
pos = nx.spring_layout(secondchoice)
color = plt.cm.Blues(np.linspace(0, 15, 1))
nx.draw_networkx_nodes(secondchoice, pos, node_size = 15000, node_color = color)
nx.draw_networkx_edges(secondchoice, pos, width = 3, alpha = 0.6, edge_color = 'brown')
nx.draw_networkx_labels(secondchoice, pos, font_size = 20, font_family = 'sans-serif')
plt.axis('off')
plt.grid()
plt.title('Top 15 drugich wyborów', fontsize = 40)
plt.show()

## Data Preprocessing

In [ ]:
# tworzenie identycznej listy dla każdego klienta
trans = []
for i in range(0, 9836 ):
     trans.append([str(data.values[i,j]) for j in range(1, 33)])

# konwertowanie go na tablicę numpy
trans = np.array(trans)

# sprawdzanie kształtu tablicy
print(trans.shape)

## Korzystanie z kodera transakcji

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
data = te.fit_transform(trans)
data = pd.DataFrame(data, columns = te.columns_)

# uzyskanie kształtu danych
data.shape

In [ ]:
# sprawdźmy kolumny

data.columns

In [ ]:
# pobranie nagłówka danych

data.head()

## Stosowanie apriori

<img src="https://blog-c7ff.kxcdn.com/blog/wp-content/uploads/2017/03/Apriori-Algorithm.jpg" width="500px">

## Jak działa algorytm Apriori?

Kluczową koncepcją algorytmu Apriori jest antymonotoniczność środka wsparcia. Zakłada to

* Wszystkie podzbiory częstego zestawu pozycji muszą być częste
* Podobnie, w przypadku każdego rzadkiego zestawu elementów, wszystkie jego super zbiory również muszą być rzadkie

**Krok 1**: Utwórz tabelę częstotliwości zawierającą wszystkie elementy występujące we wszystkich transakcjach.

**Krok 2**: Wiemy, że tylko te elementy są istotne, dla których wsparcie jest większe lub równe progowi wsparcia.
**Krok 3**: Następnym krokiem jest utworzenie wszystkich możliwych par znaczących elementów, pamiętając, że kolejność nie ma znaczenia, tj. AB jest tym samym, co BA.

**Krok 4**: Teraz policzymy wystąpienia każdej pary we wszystkich transakcjach.

**Krok 5**: Ponownie tylko te zestawy elementów są istotne, które przekraczają próg wsparcia
**Krok 6**: Powiedzmy, że chcielibyśmy poszukać zestawu trzech produktów, które są kupowane razem. Wykorzystamy zestawy przedmiotów znalezione w kroku 5 i utworzymy zestaw 3 elementów.

In [ ]:
from mlxtend.frequent_patterns import apriori

# Teraz zwróćmy przedmioty i zestawy przedmiotów z co najmniej 5% wsparciem:
apriori(data, min_support = 0.01, use_colnames = True)


Zaletą pracy z pandas DataFrames jest to, że możemy użyć wygodnych funkcji do filtrowania wyników. Na przykład, załóżmy, że interesują nas tylko zestawy elementów o długości 2, które mają podparcie co najmniej 80 procent. Najpierw tworzymy częste zestawy przedmiotów za pośrednictwem apriori i dodajemy nową kolumnę, która przechowuje długość każdego zestawu przedmiotów:

## Wybór i filtrowanie wyników

In [ ]:
frequent_itemsets = apriori(data, min_support = 0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

In [ ]:
# uzyskanie zestawów przedmiotów o długości = 2 i wsparciu powyżej 10%

frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.01) ]

In [ ]:
# uzyskanie zestawów przedmiotów o długości = 2 i wsparciu powyżej 10%

frequent_itemsets[ (frequent_itemsets['length'] == 1) &
                   (frequent_itemsets['support'] >= 0.01) ]